## Notebook Setup

In [204]:
# Import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Data Pull

In [205]:
vcl_data = pd.read_csv('../../02_data/01_raw/fraud_oracle.csv')

display(vcl_data.shape)
display(vcl_data.head())

(15420, 33)

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47,Policy Holder,Sport - Collision,Sport,more than 69000,0,3,7,400,3,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65,Third Party,Sedan - Liability,Sport,20000 to 29000,0,4,4,400,2,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,more than 69000,0,5,3,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


### Data Statistics

In [206]:
vcl_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

### Data Cleaning

#### Check Missing Values

In [228]:
# There are no columns with missing values
vcl_data.columns[vcl_data.isnull().any()]

Index([], dtype='object')

#### Check Constant Columns (columns with only 1 unique value)

In [ ]:
# There are no columns with only 1 unique value (GREAT!)

for col in [*vcl_data.columns]:
    print(f'{col}: {vcl_data[col].nunique()}')



#### Drop Irrelevant Columns

In [ ]:
# PolicyNumber is the  unique policy identifier which in our case is not relevant in predicting fraudulent claims

vcl_data = vcl_data.drop(columns = 'PolicyNumber')

#### Rename the bins of specific columns for easier sorting

In [207]:
vcl_data['PastNumberOfClaims'] = vcl_data['PastNumberOfClaims'].apply(
    lambda x: '0' if x == 'none' else
              '1' if x == '1' else
              '2 to 4' if x == '2 to 4' else
              '3' if x == '3' else
              '>4'
)

sorted(vcl_data['PastNumberOfClaims'].unique())

['0', '1', '2 to 4', '>4']

In [208]:
vcl_data['VehiclePrice'] = vcl_data['VehiclePrice'].apply(
    lambda x: '01: < 20000' if x == 'less than 20000' else
              '02: 20000 to 29000' if x == '20000 to 29000' else
              '03: 30000 to 39000' if x == '30000 to 39000' else
              '04: 40000 to 59000' if x == '40000 to 59000' else
              '05: 60000 to 69000' if x == '60000 to 69000' else
              '06: > 69000'
)

sorted(vcl_data['VehiclePrice'].unique())

['01: < 20000',
 '02: 20000 to 29000',
 '03: 30000 to 39000',
 '04: 40000 to 59000',
 '05: 60000 to 69000',
 '06: > 69000']

In [209]:
vcl_data['Days_Policy_Accident'] = vcl_data['Days_Policy_Accident'].apply(
    lambda x: '0' if x == 'none' else
              '> 30' if x == 'more than 30' else
              x
)

sorted(vcl_data['Days_Policy_Accident'].unique())

['0', '1 to 7', '15 to 30', '8 to 15', '> 30']

In [210]:
vcl_data['Days_Policy_Claim'] = vcl_data['Days_Policy_Claim'].apply(
    lambda x: '0' if x == 'none' else
              '08 to 15' if x == '8 to 15' else
              '> 30' if x == 'more than 30' else
              x
)

sorted(vcl_data['Days_Policy_Claim'].unique())

['0', '08 to 15', '15 to 30', '> 30']

In [211]:
vcl_data['AgeOfVehicle'] = vcl_data['AgeOfVehicle'].apply(
    lambda x: '01: < 2 years' if x == 'new' else
              '02: 2 years' if x == '2 years' else
              '03: 3 years' if x == '3 years' else
              '04: 4 years' if x == '4 years' else
              '05: 5 years' if x == '5 years' else
              '06: 6 years' if x == '6 years' else
              '07: 7 years' if x == '7 years' else
              '08: > 7 years' if x == 'more than 7' else
              x

)

sorted(vcl_data['AgeOfVehicle'].unique())

['01: < 2 years',
 '02: 2 years',
 '03: 3 years',
 '04: 4 years',
 '05: 5 years',
 '06: 6 years',
 '07: 7 years',
 '08: > 7 years']

In [212]:
vcl_data['AgeOfPolicyHolder'] = vcl_data['AgeOfPolicyHolder'].apply(
    lambda x: '> 65' if x == 'over 65' else
              x
)

sorted(vcl_data['AgeOfPolicyHolder'].unique())

['16 to 17',
 '18 to 20',
 '21 to 25',
 '26 to 30',
 '31 to 35',
 '36 to 40',
 '41 to 50',
 '51 to 65',
 '> 65']

In [213]:
vcl_data['NumberOfSuppliments'] = vcl_data['NumberOfSuppliments'].apply(
    lambda x: '0' if x == 'none' else
              '> 5' if x == 'more than 5' else
              x
)

sorted(vcl_data['NumberOfSuppliments'].unique())

['0', '1 to 2', '3 to 5', '> 5']

In [214]:
vcl_data['AddressChange_Claim'] = vcl_data['AddressChange_Claim'].apply(
    lambda x: '01: No change' if x == 'no change' else
              '02: < 6 months' if x == 'under 6 months' else
              '03: 1 year' if x == '1 year' else
              '04: 2 to 3 years' if x == '2 to 3 years' else
              '05: 4 to 8 years' if x == '4 to 8 years' else
              x

)

sorted(vcl_data['AddressChange_Claim'].unique())

['01: No change',
 '02: < 6 months',
 '03: 1 year',
 '04: 2 to 3 years',
 '05: 4 to 8 years']

In [215]:
vcl_data['NumberOfCars'] = vcl_data['NumberOfCars'].apply(
    lambda x: '1' if x == '1 vehicle' else
              '2' if x == '2 vehicles' else
              '> 8' if x == 'more than 8' else
              x
)

sorted(vcl_data['NumberOfCars'].unique())

['1', '2', '3 to 4', '5 to 8', '> 8']

#### Re-arrange Columns

In [217]:
# Insert the FraudFound_P column at the end of the dataframe as this is the target variable
temp = vcl_data.pop('FraudFound_P')
vcl_data.insert(vcl_data.shape[1], 'FraudFound_P', temp)

#### Review the Data Types

In [233]:
vcl_data.select_dtypes('object').columns

Index(['Month', 'DayOfWeek', 'Make', 'AccidentArea', 'DayOfWeekClaimed',
       'MonthClaimed', 'Sex', 'MaritalStatus', 'Fault', 'PolicyType',
       'VehicleCategory', 'VehiclePrice', 'Days_Policy_Accident',
       'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle',
       'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType',
       'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars',
       'BasePolicy'],
      dtype='object')

In [234]:
vcl_data.select_dtypes('int').columns

Index(['WeekOfMonth', 'WeekOfMonthClaimed', 'Age', 'RepNumber', 'Deductible',
       'DriverRating', 'Year', 'FraudFound_P'],
      dtype='object')

In [ ]:
# One-hot encoding and ordinal encoding will be applied in the feature_engineering notebook

#### Save to a .csv file

In [238]:
# path = '../../02_data/02_processed/fraud_oracle_processed.csv'
# vcl_data.to_csv(path, index = False)